In [1]:
!pip install langchain-community

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import json

# ✅ 임베딩 모델 설정 (저장 당시와 동일한 모델 이름을 써야 가장 정확)
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")

# ✅ FAISS 인덱스 로드 (index.pkl과 index.faiss 모두 자동 인식)
faiss_db = FAISS.load_local("/mnt/data", embeddings=embedding, index_name="index")

# ✅ fine-tuning 데이터 불러오기
with open("/mnt/data/fine_tuning_data_cleaned_format.json", "r", encoding="utf-8") as f:
    finetune_data = json.load(f)

# ✅ input 자동 생성
for item in finetune_data:
    if item["input"] == "":
        docs = faiss_db.similarity_search(item["instruction"], k=3)
        item["input"] = "\n\n".join([doc.page_content for doc in docs])

# ✅ 저장
output_path = "/mnt/data/fine_tuning_data_with_input.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(finetune_data, f, ensure_ascii=False, indent=2)

output_path


In [ ]:
import os
import json
import random
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate

# ✅ API 키 설정
os.environ["OPENAI_API_KEY"] = ""

# ✅ 벡터 DB 로딩
embedding = OpenAIEmbeddings()
faiss_db = FAISS.load_local("Law_case_db", embedding, index_name="index", allow_dangerous_deserialization=True)
retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# ✅ 질문 생성 프롬프트
question_prompt = PromptTemplate.from_template("""
당신은 이혼 전문 변호사입니다. 아래 사용자 상황을 바탕으로, 법률 상담을 받기 위해 사용자가 물어볼 만한 자연스러운 질문 3개를 생성해 주세요.

[사용자 상황]
{profile}

[질문]
1.
2.
3.
""")

question_gen = LLMChain(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
    prompt=question_prompt
)

# ✅ 답변 생성 체인 (RAG 기반)
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o"),
    retriever=retriever,
    return_source_documents=True
)

# ✅ 사용자 상황 템플릿
user_profiles = [
    {
        "혼인 상태": "기혼",
        "혼인 기간": "10년",
        "자녀": "2명 (고등학생, 중학생)",
        "재산": "약 5억",
        "폭력": "없음",
        "기타": "남편이 외도 중이고 주말부부임"
    },
    {
        "혼인 상태": "별거 중",
        "혼인 기간": "3년",
        "자녀": "없음",
        "재산": "1억 미만",
        "폭력": "있었음",
        "기타": "결혼 직후부터 갈등이 심했음"
    },
    {
        "혼인 상태": "사실혼",
        "혼인 기간": "5년",
        "자녀": "1명 (6세)",
        "재산": "약 2억",
        "폭력": "없음",
        "기타": "상대방이 재산을 모두 본인 명의로 함"
    }
]

# ✅ 파인튜닝 데이터 생성
def generate_finetune_data(profiles, max_samples=30):
    dataset = []
    pbar = tqdm(total=max_samples, desc="📚 Generating fine-tune data")

    for profile in profiles:
        if len(dataset) >= max_samples:
            break

        # 사용자 상황 문자열화
        profile_text = "\n".join([f"{k}: {v}" for k, v in profile.items()])

        try:
            # 질문 생성
            questions_raw = question_gen.invoke({"profile": profile_text}).get("text", "")
            questions = [
                q.strip("-•1234567890. )") for q in questions_raw.split("\n")
                if len(q.strip()) > 10
            ][:3]

            for q in questions:
                if len(dataset) >= max_samples:
                    break

                # 답변 생성 (RAG 기반)
                rag_result = qa_chain.invoke(q)
                source_docs = rag_result.get("source_documents", [])
                context = "\n\n".join([doc.page_content for doc in source_docs if hasattr(doc, "page_content")])

                # 파인튜닝 샘플 저장
                dataset.append({
                    "instruction": q,
                    "input": profile_text + "\n\n" + context.strip(),
                    "output": rag_result["result"].strip()
                })
                pbar.update(1)

        except Exception as e:
            print("⚠️ 오류:", e)
            continue

    pbar.close()
    return dataset

# ✅ 실행 및 저장
finetune_data = generate_finetune_data(user_profiles, max_samples=30)

os.makedirs("finetune_dataset", exist_ok=True)
with open("finetune_dataset/familylaw_finetune_auto.json", "w", encoding="utf-8") as f:
    json.dump(finetune_data, f, ensure_ascii=False, indent=2)

print("✅ 파인튜닝용 데이터셋 저장 완료!")

📚 Generating fine-tune data:  30%|███       | 9/30 [01:02<02:26,  6.96s/it]

✅ 파인튜닝용 데이터셋 저장 완료!


In [ ]:
import os
import json
import random
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# ✅ API 키 설정
os.environ["OPENAI_API_KEY"] = ""

# ✅ 벡터 DB 로딩
embedding = OpenAIEmbeddings()
faiss_db = FAISS.load_local("Law_case_db", embedding, index_name="index", allow_dangerous_deserialization=True)
retriever = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# ✅ 질문 생성 프롬프트
question_prompt = PromptTemplate.from_template("""
당신은 이혼 전문 변호사입니다. 아래 사용자 상황을 바탕으로, 법률 상담을 받기 위해 사용자가 물어볼 만한 자연스러운 질문 3개를 생성해 주세요.

[사용자 상황]
{profile}

[질문]
1.
2.
3.
""")

question_gen = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# ✅ 구조화된 상담 답변 생성을 위한 템플릿
structured_prompt = PromptTemplate.from_template("""
당신은 가족법 전문 AI 상담사입니다. 아래 사용자 상황과 질문, 참고 문서를 바탕으로 법적 근거가 명확한 구조화된 답변을 작성해주세요.

※ 사용자 정보 중 '비공개'로 표시된 항목은 답변에 활용하지 마세요. 또한 해당 정보가 없음을 고려해 가능한 조언을 제공해주세요.

[사용자 상황]
{profile}

[질문]
{question}

[참고 문서 요약]
{context}

[답변 작성 형식]
1️⃣ 관련 법 조항: 조문 번호와 간단한 설명 포함
2️⃣ 관련 판례 요약: 판례 번호와 핵심 내용
3️⃣ 사용자 상황에 맞춘 조언: 현실적인 설명과 권장 행동 포함

💬 답변:
""")

structured_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
qa_chain = RetrievalQA.from_chain_type(llm=structured_llm, retriever=retriever, return_source_documents=True)

# ✅ 랜덤 사용자 프로필 생성 함수
# 다루지 못하는 사용자 범위가 있어 추가 학습이 필요할 수가 있음
def generate_random_profiles(n=300):
    marital_statuses = ["기혼", "이혼", "별거", "사실혼"]
    durations = ["1년", "3년", "5년", "10년", "15년", "20년"] # 이것도 엄청 다양할 수 있음 
    divorce_stages = ["이혼 고려 중", "이혼 준비 중", "이혼 진행 중", "이미 이혼함"]
    children_opts = ["없음", "1명 (5세)", "2명 (중학생, 고등학생)", "3명 (유아/초등생/중학생)"] # 더 다양한 조합 있을 수 있음
    property_ranges = ["1천만 원 미만", "1천만~5천만 원", "5천만~1억 원", "1억~5억 원", "5억 이상"]
    abuse_history = ["없음", "있음"]

    profiles = []
    for _ in range(n):
        profile = {
            "혼인 상태": random.choice(marital_statuses) if random.random() > 0.2 else "비공개",
            "혼인 기간": random.choice(durations) if random.random() > 0.2 else "비공개",
            "이혼 단계": random.choice(divorce_stages) if random.random() > 0.2 else "비공개",
            "자녀": random.choice(children_opts) if random.random() > 0.2 else "비공개",
            "재산": random.choice(property_ranges) if random.random() > 0.2 else "비공개",
            "폭력": random.choice(abuse_history) if random.random() > 0.2 else "비공개"
        }
        profiles.append(profile)
    return profiles

# ✅ 파인튜닝 데이터 생성 함수

def generate_finetune_data(profiles, max_samples=1000):
    dataset = []
    pbar = tqdm(total=max_samples, desc="📚 Generating structured fine-tune data")

    for profile in profiles:
        if len(dataset) >= max_samples:
            break

        profile_text = " ".join([f"{k}: {v}" for k, v in profile.items()])

        try:
            q_prompt = question_prompt.format(profile=profile_text)
            questions_raw = question_gen.invoke(q_prompt).content
            questions = [
                q.strip("-•1234567890. )") for q in questions_raw.split("\n") if len(q.strip()) > 10
            ][:3]

            for question in questions:
                if len(dataset) >= max_samples:
                    break

                rag_result = qa_chain.invoke(question)
                source_docs = rag_result.get("source_documents", [])
                context_text = " ".join([doc.page_content for doc in source_docs])

                structured_prompt_input = structured_prompt.format(
                    profile=profile_text,
                    question=question,
                    context=context_text
                )
                answer = structured_llm.invoke(structured_prompt_input).content.strip()

                dataset.append({
                    "instruction": question.strip().replace("\n", " "),
                    "input": (profile_text + " " + context_text).replace("\n", " ").strip(),
                    "output": answer.replace("\n", " ").strip()
                })
                pbar.update(1)

        except Exception as e:
            print("⚠️ 오류 발생:", e)
            continue

    pbar.close()
    return dataset

# ✅ 실행 및 저장
user_profiles = generate_random_profiles(300)
finetune_data = generate_finetune_data(user_profiles, max_samples=1000)

os.makedirs("finetune_dataset", exist_ok=True)
with open("finetune_dataset/familylaw_structured_random.json", "w", encoding="utf-8") as f:
    json.dump(finetune_data, f, ensure_ascii=False, indent=2)

print("✅ 구조화된 파인튜닝 데이터 저장 완료!")
# 약 $ 2.95 사용함 

📚 Generating structured fine-tune data:  90%|█████████ | 900/1000 [2:41:55<17:59, 10.80s/it]    

✅ 구조화된 파인튜닝 데이터 저장 완료!


In [7]:
with open("finetune_dataset/user_profiles.json", "w", encoding="utf-8") as f:
    json.dump(user_profiles, f, ensure_ascii=False, indent=2)